In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

from torchmetrics.classification import BinaryAUROC, BinaryAveragePrecision
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint

In [ ]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
columns = list(X_train.columns)

In [ ]:
static_columns = columns[:25]
static_columns

In [ ]:
seq_columns = columns[25:-16]
print(seq_columns[:12])
print(seq_columns[12:19])
print(seq_columns[19:31])
print(seq_columns[31:43])
print(seq_columns[43:])

In [ ]:
goutallier_columns = columns[-16:]
goutallier_columns

In [ ]:
len(columns) == len(static_columns) + len(seq_columns) + len(goutallier_columns)

In [ ]:
label_column = "POD 6M retear"
output_columns = ["6M ASES", "6M CSS", "6M KSS", "6M VAS(activity)", "6M VAS(resting)"]
input_columns = static_columns + [column for column in seq_columns if column not in output_columns] + goutallier_columns

In [ ]:
output_columns

In [ ]:
input_columns

In [ ]:
X_train[input_columns]

In [ ]:
pd.concat([y_train, X_train[output_columns]], axis=1)

In [ ]:
def get_dataset(split):
  assert split in ["train", "val", "test"]

  X_file_name = f"X_{split}.csv"
  y_file_name = f"y_{split}.csv"

  X = pd.read_csv(X_file_name)
  y = pd.read_csv(y_file_name)

  X_np = X[input_columns].to_numpy()
  y_np = pd.concat([y, X[output_columns]], axis=1).to_numpy()

  X_tensor = torch.tensor(X_np, dtype=torch.float32)
  y_tensor = torch.tensor(y_np, dtype=torch.float32)

  return TensorDataset(X_tensor, y_tensor)

In [ ]:
trainset = get_dataset("train")
valset = get_dataset("val")
testset = get_dataset("test")

In [ ]:
in_features = len(input_columns)
out_features = len([label_column]) + len(output_columns)
in_features, out_features

In [ ]:
class MLP(L.LightningModule):
  def __init__(self, in_features, out_features):
    super().__init__()

    dropout = 0.3
    self.mlp = nn.Sequential(
      nn.Linear(in_features, 512),
      nn.LayerNorm(512),
      nn.GELU(),
      nn.Dropout(dropout),

      nn.Linear(512, 1024),
      nn.LayerNorm(1024),
      nn.GELU(),
      nn.Dropout(dropout),

      nn.Linear(1024, 512),
      nn.LayerNorm(512),
      nn.GELU(),

      nn.Linear(512, 256),
      nn.LayerNorm(256),
      nn.GELU(),

      nn.Linear(256, out_features)
    )

    self.train_roc = BinaryAUROC()
    self.val_roc = BinaryAUROC()
    self.test_roc = BinaryAUROC()
    self.test_ap = BinaryAveragePrecision()

  def forward(self, xb):
    outputs = self.mlp(xb)
    logits = outputs[:, :1]
    regs = outputs[:, 1:]
    return logits, regs

  def _shared_step(self, batch, metric=True):
    xb, yb = batch
    clf_targets = yb[:, :1]
    reg_targets = yb[:, 1:]

    logits, regs = self.forward(xb)
    clf_loss = F.binary_cross_entropy_with_logits(logits, clf_targets)
    reg_loss = F.smooth_l1_loss(regs, reg_targets)
    loss = clf_loss + reg_loss

    return {
      "loss": loss,
      "clf_loss": clf_loss,
      "reg_loss": reg_loss,
      "clf_logits": logits.detach(),
      "clf_targets": clf_targets.detach(),
    }
  
  def training_step(self, batch, batch_idx):
    out = self._shared_step(batch)

    self.log("train/loss", out["loss"], on_epoch=True, prog_bar=True)
    self.log("train/clf_loss", out["clf_loss"])
    self.log("train/reg_loss", out["reg_loss"])

    probs = out["clf_logits"].sigmoid().flatten()
    targets = out["clf_targets"].flatten().to(torch.int)
    self.train_roc.update(probs, targets)

    return out["loss"]

  def validation_step(self, batch, batch_idx):
    out = self._shared_step(batch)

    self.log("val/loss", out["loss"], prog_bar=True)
    self.log("val/clf_loss", out["clf_loss"])
    self.log("val/reg_loss", out["reg_loss"])

    probs = out["clf_logits"].sigmoid().flatten()
    targets = out["clf_targets"].flatten().to(torch.int)
    self.val_roc.update(probs, targets)
    
    return out["loss"]

  def test_step(self, batch, batch_idx):
    out = self._shared_step(batch)

    self.log("test/loss", out["loss"], prog_bar=True)
    self.log("test/clf_loss", out["clf_loss"])
    self.log("test/reg_loss", out["reg_loss"])

    probs = out["clf_logits"].sigmoid().flatten()
    targets = out["clf_targets"].flatten().to(torch.int)
    self.test_roc.update(probs, targets)
    self.test_ap.update(probs, targets)
    
    return out["loss"]
  
  def on_train_epoch_end(self):
    self.log("train/roc", self.train_roc.compute())
    self.train_roc.reset()

  def on_validation_epoch_end(self):
    self.log("val/roc", self.val_roc.compute())
    self.val_roc.reset()

  def on_test_epoch_end(self):
    self.log("test/roc", self.test_roc.compute())
    self.log("test/ap", self.test_ap.compute())
    self.test_roc.reset()

  def configure_optimizers(self):
    return optim.AdamW(self.parameters(), lr=5e-5, weight_decay=1e-4)

In [ ]:
# val_logs = []
# batch_size = 128
# num_experiments = 10
# for i in range(num_experiments):
#   mlp = MLP(in_features, out_features)
#   trainloader = DataLoader(trainset, batch_size=batch_size)
#   valloader = DataLoader(valset, batch_size=batch_size)

#   trainer = L.Trainer(max_epochs=10)
#   trainer.fit(mlp, trainloader, valloader)
#   val_logs.append(trainer.test(mlp, valloader))

In [ ]:
models = []
val_logs = []

batch_size = 128
max_epochs = 80
num_experiments = 5

for i in range(num_experiments):
  mlp = MLP(in_features, out_features)
  trainloader = DataLoader(trainset, batch_size=batch_size)
  valloader  = DataLoader(valset,  batch_size=batch_size)

  trainer = L.Trainer(
    max_epochs=80,
    callbacks=[ModelCheckpoint(monitor='val/roc', mode='max', save_top_k=1)]
  )
  trainer.fit(mlp, trainloader, valloader)
  models.append(mlp)
  val_logs.append(trainer.test(mlp, valloader))

In [ ]:
val_aps = np.array([val_log[0]["test/ap"] for val_log in val_logs])
val_rocs = np.array([val_log[0]["test/roc"] for val_log in val_logs])
pd.DataFrame({"ROC AUC": val_rocs, "PR AUC": val_aps}).describe()

In [ ]:
# def get_best_model():
best_roc_idx = val_rocs.argmax()
best_mlp = models[best_roc_idx]

In [ ]:
def show_set_stat(dataset):
  _, y = dataset[:]
  negative, positive = torch.bincount(y[:, 0].to(torch.int)).tolist()
  samples = len(dataset)

  print(f"tatal   : {samples}")
  print(f"negative: {negative:3} ({negative/samples*100:5.2f}%)")
  print(f"positive: {positive:3} ({positive/samples*100:5.2f}%)")

In [ ]:
print("trainset (SMOTE)")
show_set_stat(trainset)

In [ ]:
print("valset")
show_set_stat(valset)

In [ ]:
@torch.no_grad()
def forward_loader(model, dataloader):
  all_logits = []
  all_regs = []
  all_clf_targets = []
  all_reg_targets = []
  
  model.eval()
  for xb, yb in dataloader:
    logits, regs = mlp(xb)
    all_logits.append(logits)
    all_regs.append(regs)
    all_clf_targets.append(yb[:, :1])
    all_reg_targets.append(yb[:, 1:])

  logits = torch.cat(all_logits).flatten()
  regs = torch.cat(all_regs)
  clf_targets = torch.cat(all_clf_targets).to(torch.int).flatten()
  reg_targets = torch.cat(all_reg_targets)

  return logits, regs, clf_targets, reg_targets

In [ ]:
logits, regs, clf_targets, reg_targets = forward_loader(best_mlp, valloader)
probs = logits.sigmoid()

print(f"logits.shape:      {logits.shape}")
print(f"probs.shape:       {probs.shape}")
print(f"regs.shape:        {regs.shape}")
print()
print(f"clf_targets.shape: {clf_targets.shape}")
print(f"reg_targets.shape: {reg_targets.shape}")

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(clf_targets, probs)
thresholds = np.append(thresholds, 1.0)

plt.figure(figsize=(8, 6))
plt.plot(thresholds, precisions, label='Precision', marker='o', markersize=3)
plt.plot(thresholds, recalls, label='Recall', marker='x', markersize=3)

plt.title("Precision & Recall vs Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
def plot_score_distributions(
  y_score, y_true, *,
  bins=40,
  title=None,
  density=False,
  th_lines=(0.5,),
):
  y_true = np.asarray(y_true).astype(int)
  y_score = np.asarray(y_score)

  x_main = y_score
  x_label = "Predicted probability"

  pos = x_main[y_true == 1]
  neg = x_main[y_true == 0]

  xmin = np.min(x_main)
  xmax = np.max(x_main)
  bins_edges = np.linspace(xmin, xmax, bins+1)

  plt.figure(figsize=(9, 5.5))
  plt.hist(neg, bins=bins_edges, alpha=0.55, density=density,
           label=f"Negative (n={len(neg)})", edgecolor="white", linewidth=0.5)
  plt.hist(pos, bins=bins_edges, alpha=0.55, density=density,
           label=f"Positive (n={len(pos)})", edgecolor="white", linewidth=0.5)

  if th_lines:
    for th in th_lines:
      plt.axvline(th, linestyle="--", linewidth=1.5)

  plt.xlabel(x_label)
  plt.ylabel("Density" if density else "Count")
  plt.title(title or "Score distributions by class")
  plt.legend(loc="best")
  plt.grid(True, linestyle="--", alpha=0.4)

  plt.tight_layout()
  plt.show()

In [ ]:
default_thresholds = np.linspace(0, 1, 11)[1:-1].tolist() # [0.1, 0.2, ... , 0.9]

def test_thresholds(y_score, y_true, thresholds=default_thresholds, verbose=True):
  accuracies = []
  precisions = []
  recalls = []
  f1s = []
  for threshold in thresholds:
    bin_acc = BinaryAccuracy(threshold)
    bin_precison = BinaryPrecision(threshold)
    bin_recall = BinaryRecall(threshold)
    bin_f1 = BinaryF1Score(threshold)

    bin_acc.update(y_score, y_true)
    bin_precison.update(y_score, y_true)
    bin_recall.update(y_score, y_true)
    bin_f1.update(y_score, y_true)

    accuracies.append(bin_acc.compute().item())
    precisions.append(bin_precison.compute().item())
    recalls.append(bin_recall.compute().item())
    f1s.append(bin_f1.compute().item())

  result = pd.DataFrame({
    "threshold": thresholds,
    "accuracy": accuracies,
    "precison": precisions,
    "recall": recalls,
    "f1": f1s
  }).set_index("threshold")

  if verbose:
    print(result)

  return result

In [ ]:
thresholds = [0.5399]
test_thresholds(probs, clf_targets, thresholds)
plot_score_distributions(probs, clf_targets, bins=40, density=False, th_lines=thresholds)

In [ ]:
print(testset)
show_set_stat(testset)

In [ ]:
testloader = DataLoader(testset, batch_size=batch_size)
test_logs = trainer.test(mlp, testloader)

In [ ]:
test_logits, test_regs, test_clf_targets, test_reg_targets = forward_loader(mlp, testloader)
test_probs = test_logits.sigmoid()
test_thresholds(test_probs, test_clf_targets, thresholds)
plot_score_distributions(test_probs, test_clf_targets, bins=40, density=True, th_lines=thresholds)

In [ ]:
pre_columns = seq_columns[:12] + goutallier_columns[:4] + goutallier_columns[8:12]
pre_columns

In [ ]:
mean_columns = [column for column in columns if column not in static_columns + pre_columns + output_columns]
mean_columns

In [ ]:
mean_table = pd.read_csv("X_train.csv")
mean_table["age_group"] = mean_table["나이"] // 10 * 10

group_columns = ["성별 (M:1,F:2)", "age_group"]
mean_table = mean_table.groupby(group_columns)[mean_columns].mean().reset_index()
mean_table

In [ ]:
def get_pre_with_mean_dataset(split):
  assert split in ["val", "test"]
  X = pd.read_csv(f"X_{split}.csv")
  y = pd.read_csv(f"y_{split}.csv")

  indices = pd.concat([X["성별 (M:1,F:2)"], X["나이"] // 10 * 10], axis=1)
  indices.columns = group_columns
  mean_values = indices.merge(mean_table, on=group_columns, how="left")

  X[mean_columns] = mean_values[mean_columns]
  X_np = X[input_columns].to_numpy()
  y_np = pd.concat([y, X[output_columns]], axis=1).to_numpy()

  X_tensor = torch.tensor(X_np, dtype=torch.float32)
  y_tensor = torch.tensor(y_np, dtype=torch.float32)

  return TensorDataset(X_tensor, y_tensor)

In [ ]:
val_pre_with_mean_set = get_pre_with_mean_dataset("val")
val_pre_with_mean_loader = DataLoader(val_pre_with_mean_set, batch_size=batch_size)
val_pre_with_mean_logs = trainer.test(mlp, val_pre_with_mean_loader)

In [ ]:
logits, regs, clf_targets, reg_targets = forward_loader(mlp, val_pre_with_mean_loader)
probs = logits.sigmoid()
test_thresholds(probs, clf_targets, thresholds)
plot_score_distributions(probs, clf_targets, bins=40, density=True, th_lines=thresholds)

In [ ]:
def fgsm_attack(data, data_grad, epsilon):
  sign_data_grad = data_grad.sign()
  perturbed_data = data + epsilon*sign_data_grad
  return perturbed_data

In [ ]:
num_static_columns = len(static_columns)
num_goutallier_columns= len(goutallier_columns)
num_static_columns, num_goutallier_columns

In [ ]:
# fgsm_target_start = num_static_columns+12
# fgsm_target_end = -num_goutallier_columns
# fgsm_target_columns = input_columns[fgsm_target_start:fgsm_target_end]
# fgsm_target_columns, len(fgsm_target_columns)

In [ ]:
fgsm_target_indices = []
fgsm_target_columns = []
fgsm_target_features = ["ERabd", "ERside", "FF", "IR", "MMTgrade", "MMTsec", "add"]

for idx, column in enumerate(input_columns):
  for feature in fgsm_target_features:
    if feature in column and "0M" not in column:
      break
  else:
    continue

  fgsm_target_indices.append(idx)
  fgsm_target_columns.append(column)

[(idx, column) for idx, column in zip(fgsm_target_indices, fgsm_target_columns)]

In [ ]:
# # 0: '6M ASES'          -> maximize
# # 1: '6M CSS'           -> maximize
# # 2: '6M KSS'           -> maximize
# # 3: '6M VAS(activity)' -> minimize
# # 4: '6M VAS(resting)'  -> minimize
# maximize_indices = [0, 1, 2]
# minimize_indices = [3, 4]

# lambda_logits = 1.0
# lambda_reg = 0.3
# epsilon = 0.1
# num_fgsm = 10

# all_logits = []
# all_regs = []
# all_perturbed_xb = []
# all_perturbed_logits = []
# all_perturbed_regs = []

# mlp.eval()
# for xb, yb in val_pre_with_mean_loader:
#   clf_targets = yb[:, :1]
#   reg_targets = yb[:, 1:]

#   xb.requires_grad = True
#   xb_target = xb[:, fgsm_target_indices]
#   logits, regs = mlp(xb)
#   all_logits.append(logits.detach())
#   all_regs.append(regs.detach())

#   clf_loss = F.binary_cross_entropy_with_logits(logits, clf_targets)
#   logits_dir_loss = -logits.mean()

#   reg_inc_term = -regs[:, maximize_indices].mean()
#   reg_dec_term = regs[:, minimize_indices].mean()
#   reg_dir_loss = reg_inc_term + reg_dec_term

#   loss = clf_loss + lambda_logits * logits_dir_loss + lambda_reg * reg_dir_loss

#   mlp.zero_grad()
#   loss.backward()

#   xb_target_grad = xb.grad.data[:, fgsm_target_indices]
#   perturbed_xb_target = fgsm_attack(xb_target, xb_target_grad, epsilon)

#   perturbed_xb = xb.detach().clone()
#   perturbed_xb[:, fgsm_target_indices] = perturbed_xb_target
#   all_perturbed_xb.append(perturbed_xb.detach())

#   perturbed_logits, perturbed_regs = mlp(perturbed_xb)
#   all_perturbed_logits.append(perturbed_logits.detach())
#   all_perturbed_regs.append(perturbed_regs.detach())

In [ ]:
# # 0: '6M ASES'          -> maximize
# # 1: '6M CSS'           -> maximize
# # 2: '6M KSS'           -> maximize
# # 3: '6M VAS(activity)' -> minimize
# # 4: '6M VAS(resting)'  -> minimize
maximize_indices = [0, 1, 2]
minimize_indices = [3, 4]

lambda_logits = 1.0
lambda_reg = 0.3
epsilon = 1e-2
num_fgsm_attack = 50

all_logits = []
all_regs = []
all_perturbed_xb = []
all_perturbed_logits = []
all_perturbed_regs = []

mlp.eval()
for xb, yb in val_pre_with_mean_loader:
  clf_targets = yb[:, :1]
  reg_targets = yb[:, 1:]

  current_xb = xb.detach().clone()
  current_xb.requires_grad = True

  logits, regs = mlp(current_xb)
  all_logits.append(logits.detach())
  all_regs.append(regs.detach())

  for _ in range(num_fgsm_attack):
    clf_loss = F.binary_cross_entropy_with_logits(logits, clf_targets)

    logits_dir_loss = -logits.mean()

    reg_inc_term = regs[:, maximize_indices].mean()
    reg_dec_term = -regs[:, minimize_indices].mean()
    reg_dir_loss = reg_inc_term + reg_dec_term

    loss = clf_loss + lambda_logits * logits_dir_loss + lambda_reg * reg_dir_loss

    mlp.zero_grad()
    if current_xb.grad is not None:
      current_xb.grad.zero_()

    loss.backward()

    xb_target_grad = current_xb.grad.data[:, fgsm_target_indices]
    xb_target = current_xb.detach()[:, fgsm_target_indices]

    perturbed_xb_target = fgsm_attack(xb_target, xb_target_grad, epsilon)

    updated_xb = current_xb.detach().clone()
    updated_xb[:, fgsm_target_indices] = perturbed_xb_target

    current_xb = updated_xb.detach().clone()
    current_xb.requires_grad = True
    logits, regs = mlp(current_xb)

  all_perturbed_xb.append(current_xb.detach())

  final_logits = logits
  final_regs = regs

  all_perturbed_logits.append(final_logits.detach())
  all_perturbed_regs.append(final_regs.detach())

In [ ]:
logits = torch.cat(all_logits).flatten()
regs = torch.cat(all_regs)
logits.shape, regs.shape

In [ ]:
perturbed_logits = torch.cat(all_perturbed_logits).flatten()
perturbed_regs = torch.cat(all_perturbed_regs)
perturbed_logits.shape, perturbed_regs.shape

In [ ]:
probs = logits.sigmoid()
perturbed_probs = perturbed_logits.sigmoid()
probs.shape, perturbed_probs.shape

In [ ]:
print(label_column)
prob_results = pd.DataFrame({
  "probability": probs.flatten(),
  "after probability": perturbed_probs.flatten()
})
prob_results["delta"] = prob_results["after probability"] - prob_results["probability"]
prob_results

In [ ]:
prob_results.describe()

In [ ]:
all_feature_results = []
for feature_idx in range(regs.size(1)):
  feature_column = output_columns[feature_idx]
  after_column = f"after {feature_column}"

  feature_results = pd.DataFrame({
    feature_column: regs[:, feature_idx],
    after_column: perturbed_regs[:, feature_idx]
  })
  feature_results["delta"] = feature_results[after_column] - feature_results[feature_column]
  all_feature_results.append(feature_results)

  direction = "maximize" if feature_idx in maximize_indices else "minimize"
  print(f"{feature_column} ({direction})")
  print(feature_results)
  print()

In [ ]:
for feature_idx, (feature_column, feature_results) in enumerate(zip(output_columns, all_feature_results)):
  direction = "maximize" if feature_idx in maximize_indices else "minimize"
  print(f"{feature_column} ({direction})")
  print(feature_results.describe())
  print()

In [ ]:
perturbed_xbs = torch.cat(all_perturbed_xb)[:, fgsm_target_indices]
perturbed_xbs.shape

In [ ]:
all_xb = []
for xb, yb in val_pre_with_mean_loader:
  all_xb.append(xb[:, fgsm_target_indices])
xbs = torch.cat(all_xb)
xbs.shape

In [ ]:
xbs = xbs.cpu().detach().numpy()
perturbed_xbs = perturbed_xbs.cpu().detach().numpy()

In [ ]:
delta = perturbed_xbs - xbs  # [batch, features]

df_before = pd.DataFrame(xbs, columns=fgsm_target_columns)
df_after = pd.DataFrame(perturbed_xbs, columns=fgsm_target_columns)
df_delta = pd.DataFrame(delta, columns=fgsm_target_columns)

mean_before = df_before.mean(axis=0)
mean_after = df_after.mean(axis=0)
mean_delta = df_delta.mean(axis=0)

x = np.arange(len(fgsm_target_columns))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, mean_before.values, width, label='before')
plt.bar(x + width/2, mean_after.values, width, label='after')
plt.xticks(x, fgsm_target_columns, rotation=45, ha='right')
plt.ylabel("Mean value across batch")
plt.title("Feature-wise mean before vs after FGSM")
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 5))
plt.bar(x, mean_delta.values)
plt.xticks(x, fgsm_target_columns, rotation=45, ha='right')
plt.axhline(0, linewidth=1)
plt.ylabel("Mean Δ (after - before)")
plt.title("Feature-wise mean change after FGSM")
plt.tight_layout()
plt.show()

summary_df = pd.DataFrame({
  "mean_before": mean_before,
  "mean_after": mean_after,
  "mean_delta": mean_delta
})
print(summary_df)

In [ ]:
import joblib
from sklearn.preprocessing import StandardScaler

scaler = joblib.load("scaler.pkl")
scaler: StandardScaler

In [ ]:
scaler_columns = list(scaler.get_feature_names_out())
scaler_columns

In [ ]:
n_scaled_samples = perturbed_xbs.shape[0]
n_scaled_samples

In [ ]:
def inverse_scale(scaler, xb):
  n_scaled_samples = xb.shape[0]
  scaled_data = pd.DataFrame(np.zeros((n_scaled_samples, len(scaler_columns))), columns=scaler_columns)
  scaled_data[fgsm_target_columns] = xb
  inversed_data = scaler.inverse_transform(scaled_data)
  inversed_data = pd.DataFrame(inversed_data, columns=scaler_columns)
  return inversed_data[fgsm_target_columns]

In [ ]:
perturbed_examples = inverse_scale(scaler, perturbed_xbs)
perturbed_examples

In [ ]:
original_examples = inverse_scale(scaler, xbs)
original_examples

In [ ]:
perturbed_examples.to_csv("perturbed_examples.csv", index=False)
original_examples.to_csv("original_examples.csv", index=False)

In [ ]:
probs.shape, regs.shape

In [ ]:
perturbed_probs.shape, perturbed_probs.shape

In [ ]:
scaler.get_feature_names_out()

In [ ]:
def inverse_regs(scaler, regs):
  n_scaled_samples = regs.shape[0]
  scaled_data = pd.DataFrame(np.zeros((n_scaled_samples, len(scaler_columns))), columns=scaler_columns)
  scaled_data[output_columns] = regs
  inversed_data = scaler.inverse_transform(scaled_data)
  inversed_data = pd.DataFrame(inversed_data, columns=scaler_columns)
  return inversed_data[output_columns]

In [ ]:
original_regs_df = inverse_regs(scaler, regs)
original_regs_df

In [ ]:
perturbed_regs_df = inverse_regs(scaler, perturbed_regs)
perturbed_regs_df

In [ ]:
original_probs_df = pd.DataFrame(probs, columns=[label_column])
original_probs_df

In [ ]:
perturbed_probs_df = pd.DataFrame(perturbed_probs, columns=[label_column])
perturbed_probs_df

In [ ]:
original_outputs = pd.concat([original_probs_df, original_regs_df], axis=1)
original_outputs

In [ ]:
perturbed_outputs = pd.concat([perturbed_probs_df, perturbed_regs_df], axis=1)
perturbed_outputs

In [ ]:
original_outputs.to_csv("original_outputs.csv", index=False)
perturbed_outputs.to_csv("perturbed_outputs.csv", index=False)

In [ ]:
_, y_val = val_pre_with_mean_set[:]
pos_indicies = (y_val[:, :1] == 1.).flatten().numpy()

In [ ]:
original_examples[pos_indicies]

In [ ]:
perturbed_examples[pos_indicies]

In [ ]:
original_outputs[pos_indicies]

In [ ]:
perturbed_outputs[pos_indicies]

In [ ]:
x = np.arange(len(fgsm_target_columns))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, mean_before.values, width, label='before')
plt.bar(x + width/2, mean_after.values, width, label='after')
plt.xticks(x, fgsm_target_columns, rotation=45, ha='right')
plt.ylabel("Mean value across batch")
plt.title("Feature-wise mean before vs after FGSM")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
result_columns = fgsm_target_columns + [label_column] + output_columns 
result_columns

In [ ]:
idx = 129

In [ ]:
original_examples.loc[[idx], :]

In [ ]:
original_outputs.loc[[idx], :]

In [ ]:
before = pd.concat([
  original_examples.loc[[idx], :],
  original_outputs.loc[[idx], :]
], axis=1)
before

In [ ]:
after = pd.concat([
  perturbed_examples.loc[[idx], :],
  perturbed_outputs.loc[[idx], :]
], axis=1)
after

In [ ]:
pd.concat([before, after], axis=0).to_csv("before_after.csv", index=False)

In [ ]:
x = np.arange(len(result_columns[:-6]))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, before.values.ravel()[:-6], width, label='before')
plt.bar(x + width/2, after.values.ravel()[:-6], width, label='after')
plt.xticks(x, result_columns[:-6], rotation=45, ha='right')
plt.title(f"Before vs After FGSM (index={idx})")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
x = np.arange(len(result_columns[-6:]))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, before.values.ravel()[-6:], width, label='before')
plt.bar(x + width/2, after.values.ravel()[-6:], width, label='after')
plt.xticks(x, result_columns[-6:], rotation=45, ha='right')
plt.title(f"Before vs After FGSM (index={idx})")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
x = np.arange(len(result_columns[-6:-5]))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, before.values.ravel()[-6:-5], width, label='before')
plt.bar(x + width/2, after.values.ravel()[-6:-5], width, label='after')
plt.xticks(x, result_columns[-6:-5], rotation=45, ha='right')
plt.title(f"Before vs After FGSM (index={idx})")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
x = np.arange(len(result_columns[-5:]))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, before.values.ravel()[-5:], width, label='before')
plt.bar(x + width/2, after.values.ravel()[-5:], width, label='after')
plt.xticks(x, result_columns[-5:], rotation=45, ha='right')
plt.title(f"Before vs After FGSM (index={idx})")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
@torch.no_grad()
def forward_loader(model, dataloader):
  all_logits, all_regs, all_clf_targets, all_reg_targets = [], [], [], []
  model.eval()
  for xb, yb in dataloader:
    logits, regs = model(xb)
    all_logits.append(logits)
    all_regs.append(regs)
    all_clf_targets.append(yb[:, :1])
    all_reg_targets.append(yb[:, 1:])
  logits = torch.cat(all_logits).flatten()
  regs = torch.cat(all_regs)
  clf_targets = torch.cat(all_clf_targets).to(torch.int).flatten()
  reg_targets = torch.cat(all_reg_targets)
  return logits, regs, clf_targets, reg_targets


In [ ]:
import re

mono_groups = {}
pattern = re.compile(r"(\d+)M\s+(.+)")
for idx, col in enumerate(input_columns):
  m = pattern.match(col)
  if m is None: 
    continue
  month = int(m.group(1))
  feat  = m.group(2).strip()
  if idx not in fgsm_target_indices:
    continue
  mono_groups.setdefault(feat, []).append((month, idx))

for feat in mono_groups:
    mono_groups[feat] = sorted(mono_groups[feat], key=lambda x: x[0])

increasing_features = {"ERabd", "ERside", "FF", "IR", "MMTgrade", "MMTsec", "add"}
decreasing_features = {"VAS"}

def feature_direction(feat: str) -> str:
  base = feat
  if any(k in base for k in increasing_features):
    return "inc"
  if any(k in base for k in decreasing_features):
    return "dec"
  return "inc"

def monotonic_and_smoothness_penalty(xb, margin=0.0):
  mono_terms = []
  smooth_terms = []

  for feat, seq in mono_groups.items():
    if len(seq) < 2:
      continue
    dirn = feature_direction(feat)
    idxs = [col_idx for (_, col_idx) in seq]
    xs = xb[:, idxs] # [B, T]

    diffs = xs[:, 1:] - xs[:, :-1] # [B, T-1]
    if dirn == "inc":
      mono = F.relu(-diffs + margin).mean()
    else:
      mono = F.relu(diffs + margin).mean()
    mono_terms.append(mono)

    if xs.size(1) >= 3:
      x_tminus1 = xs[:, :-2]
      x_t = xs[:, 1:-1]
      x_tplus1 = xs[:, 2:]
      second_diff = (x_tplus1 - 2*x_t + x_tminus1) # [B, T-2]
      smooth = (second_diff ** 2).mean()
      smooth_terms.append(smooth)

  mono_loss = torch.stack(mono_terms).mean() if mono_terms else xb.new_tensor(0.0)
  smooth_loss = torch.stack(smooth_terms).mean() if smooth_terms else xb.new_tensor(0.0)
  return mono_loss, smooth_loss

epsilon_total = 0.08    
step_size = 0.004   
pgd_steps = 50

lambda_logits = 1.0 
lambda_regdir = 0.3 
lambda_mono = 0.5 
lambda_smooth = 0.1 
mono_margin = 0.00

std_clip_min = -4.0
std_clip_max =  4.0

mlp.eval()
all_pgd_xb = []
all_pgd_logits = []
all_pgd_regs = []

for xb, yb in val_pre_with_mean_loader:
  clf_targets = yb[:, :1]
  reg_targets = yb[:, 1:]

  original_xb = xb.detach().clone()
  current_xb = xb.detach().clone().requires_grad_(True)

  logits, regs = mlp(current_xb)

  for _ in range(pgd_steps):
    clf_loss = F.binary_cross_entropy_with_logits(logits, clf_targets)
    logits_dir_loss = -logits.mean()
    reg_inc_term = +regs[:, maximize_indices].mean() 
    reg_dec_term = -regs[:, minimize_indices].mean() 
    reg_dir_loss = reg_inc_term + reg_dec_term

    mono_loss, smooth_loss = monotonic_and_smoothness_penalty(current_xb, margin=mono_margin)
    loss = clf_loss + lambda_logits*logits_dir_loss + lambda_regdir*reg_dir_loss + lambda_mono*mono_loss + lambda_smooth*smooth_loss

    mlp.zero_grad()
    if current_xb.grad is not None:
      current_xb.grad.zero_()
    loss.backward()

    with torch.no_grad():
      grad = current_xb.grad
      delta = torch.zeros_like(current_xb)
      delta[:, fgsm_target_indices] = step_size * grad[:, fgsm_target_indices].sign()

      updated = current_xb + delta

      diff = torch.clamp(updated - original_xb, min=-epsilon_total, max=epsilon_total)
      projected = original_xb + diff
      projected = torch.clamp(projected, std_clip_min, std_clip_max)
      current_xb = projected.detach().clone().requires_grad_(True)
      logits, regs = mlp(current_xb)

  all_pgd_xb.append(current_xb.detach())
  all_pgd_logits.append(logits.detach())
  all_pgd_regs.append(regs.detach())

pgd_perturbed_xb  = torch.cat(all_pgd_xb)
pgd_perturbed_log = torch.cat(all_pgd_logits).flatten()
pgd_perturbed_reg = torch.cat(all_pgd_regs)

In [ ]:
import matplotlib.pyplot as plt

def plot_feature_mean_trajectories(xb_dict):
  for feat, seq in mono_groups.items():
    months = [m for (m, _) in seq]
    idxs   = [idx for (_, idx) in seq]
    plt.figure(figsize=(6,4))
    for label, arr in xb_dict.items():
      xs = arr[:, idxs].cpu().numpy()  # [N, T]
      mean = xs.mean(axis=0)
      std  = xs.std(axis=0)
      plt.plot(months, mean, marker='o', label=label)
      plt.fill_between(months, mean-std, mean+std, alpha=0.2)
    plt.title(f"{feat} mean")
    plt.xlabel("Month")
    plt.ylabel("Value (std)")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_feature_mean_trajectories({
  "original": orig_xb,
  "pgd": pgd_perturbed_xb,
})

In [ ]:
def monotone_rate(xb):
  rates = {}
  for feat, seq in mono_groups.items():
    if len(seq) < 2: 
      continue
    idxs = [col_idx for (_, col_idx) in seq]
    xs = xb[:, idxs].cpu().numpy()  # [N, T]
    dirn = feature_direction(feat)
    ok = 0
    for row in xs:
      diffs = np.diff(row)
      if dirn == "inc":
        if np.all(diffs >= -1e-8):  
          ok += 1
      else:
        if np.all(diffs <=  1e-8):
          ok += 1
    rates[feat] = ok / xs.shape[0]
  return rates

def total_variation(xb):
  tvs = {}
  for feat, seq in mono_groups.items():
    if len(seq) < 2: continue
    idxs = [col_idx for (_, col_idx) in seq]
    xs = xb[:, idxs].cpu().numpy()
    tvs[feat] = np.mean(np.sum(np.abs(np.diff(xs, axis=1)), axis=1))
  return tvs

def curvature_penalty(xb):
  curvs = {}
  for feat, seq in mono_groups.items():
    if len(seq) < 3: continue
    idxs = [col_idx for (_, col_idx) in seq]
    xs = xb[:, idxs].cpu().numpy()
    sd = xs[:, 2:] - 2*xs[:, 1:-1] + xs[:, :-2]
    curvs[feat] = float(np.mean(np.sum(sd**2, axis=1)))
  return curvs


orig_xb = torch.cat([xb for xb, _ in val_pre_with_mean_loader])  
metrics_df = []
for tag, arr in [
  ("original", orig_xb),
  ("pgd",      pgd_perturbed_xb),
  ("pgd+iso",  pgd_iso),
]:
  mr = monotone_rate(arr)
  tv = total_variation(arr)
  cv = curvature_penalty(arr)
  for feat in mono_groups.keys():
    row = {
      "version": tag,
      "feature": feat,
      "monotone_rate": mr.get(feat, np.nan),
      "total_variation": tv.get(feat, np.nan),
      "curvature": cv.get(feat, np.nan),
    }
    metrics_df.append(row)

metrics_df = pd.DataFrame(metrics_df)


In [ ]:
from sklearn.isotonic import IsotonicRegression

def isotonic_fix(sequence, direction="inc"):
  x = np.arange(len(sequence))
  y = sequence
  if direction == "inc":
    ir = IsotonicRegression(increasing=True, out_of_bounds="clip")
    return ir.fit_transform(x, y)
  else:
    ir = IsotonicRegression(increasing=False, out_of_bounds="clip")
    return ir.fit_transform(x, y)

pgd_iso = pgd_perturbed_xb.clone().cpu().numpy()
for feat, seq in mono_groups.items():
  idxs = [col_idx for (_, col_idx) in seq]
  dirn = feature_direction(feat)
  ys = pgd_iso[:, idxs]  # [N, T]
  for i in range(ys.shape[0]):
    ys[i, :] = isotonic_fix(ys[i, :], direction=dirn)
  pgd_iso[:, idxs] = ys

pgd_iso = torch.tensor(pgd_iso, dtype=torch.float32)

In [ ]:

plot_feature_mean_trajectories({
  "original": orig_xb,
  "pgd": pgd_perturbed_xb,
  "pgd+iso": pgd_iso,
})


In [ ]:

plot_feature_mean_trajectories({
  "original": orig_xb,
  "pgd": pgd_perturbed_xb,
  "pgd+iso": pgd_iso,
})
